<img src="images/Picture0.png" width=200x />

# Notebook 08 - A Short Primer on Debugging
Code development is a rewarding experience which can provide great short-term reward. On the flip side, code which does not behave as anticipated can be an extremely frustrating experience. And there we are, debugging and starting to brain-loop over our own code. This section will provide few tips and pointers on how to investigate your faulty code and hopefully reduce your frustrations.

## Instructions
Read the material below and complete the exercises.

Material covered in this notebook:

- How to call the Python degugger
- Basic commands in the debugger
- What to do after importing faulty module 

### Credits
- Original version from Martin-D.Lacasse @ INMAS
- Dice game example from [pdb_tutorial](https://github.com/spiside/pdb-tutorial) under MIT license


## The Poorman Debugger
A very common approach to debugging is to scatter `print()` statements all over your code. Python is great for that purpose as it natively understands how to print almost any object it creates. This approach is easy, but not particularly efficient as one deals with large arrays. We'll try to cover a few scenarios and give you a few better strategies for debugging your code.

## Debugging modules
When debugging modules, it is important to import them again after changes are made. Unfortunately, the way to reload modules has changed a few times and evolved depending on which version of Python you are running. For Python version after 3.4, this is done using the `importlib` module as `import` will not re-import a module already imported. Let's see how this works in practice with a very simple example. Another way to re-import a module is to restart your kernel all together, but this is not always practical.

First we will learn how to import modules from subdirectories. Here, the module *faultyModule* is located in a directory called *Code_08*. Just notice how the directory becomes considered as a unifying module.

In [1]:
from Code_08 import faultyModule

We can investigate what is in this module using the `dir()` function and the `__doc__` strings.

In [2]:
dir(faultyModule)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'computePi']

In [3]:
print(faultyModule.__doc__)
print(faultyModule.computePi.__doc__)


A faulty module to demonstrate the use of importlib.reload()

Compute pi/4 using arctan() Taylor series and print Pi on std output


Now let's run the only function in this module.

In [4]:
faultyModule.computePi()

UnboundLocalError: cannot access local variable 'p' where it is not associated with a value

Oops! Now edit the code in another Jupyter tab and fix the problem. First `import` and try to run again. It won't work. Try `reload` and run again.

In [ ]:
# import importlib
# importlib.reload(Code_08/faultyModule)
from Code_08 import faultyModule
faultyModule.computePi()

This was an easy bug to find as Python's run-time diagnostics have significantly improved with version number. Other situations require to slowly go through the code and interrogate the various variables. This can be done with `pdb`, the Python debugger which we will cover shortly.

Another common problem with Python is when a module is not found while importing it. This can be diagnosed using the following code before the import statement.

In [ ]:
import sys
import os
# print(sys.path)
for dir in sys.path:
    print('"'+dir+'"', os.path.isfile(os.path.join(dir, 'Code_08/faultyModule.py')))

## Debugging code


The Python debugger can be run interactively, or through Jupyter as

In [5]:
%pdb on

Automatic pdb calling has been turned ON


From now on, the Python debugger will be called when an exception is raised, or when a breakpoint is set. We will now go though the exercise of debugging a game written by Spiro Sideris to learn arithmetics by adding values on a series of dice. Let's play it a few runs until...

In [6]:
%run Code_08/main.py

Add the values of the dice
It's really that easy
What are you doing with your life.
Round 1

---------
|*     *|
|*     *|
|*     *|
---------
---------
|*      |
|   *   |
|      *|
---------
---------
|*     *|
|   *   |
|*     *|
---------
---------
|*     *|
|*     *|
|*     *|
---------
---------
|*      |
|   *   |
|      *|
---------
Sigh. What is your guess?: 24
Sorry that's wrong
The answer is: 5
Like seriously, how could you mess that up
Wins: 0 Loses 1
Would you like to play again?[Y/n]: n


UnnecessaryError: You actually called this function...

> c:\users\marti\onedrive\documents\jhu\inmas\workshops\1- python\final 2022\github\code_08\dicegame\utils.py(11)i_just_throw_an_exception()
      7     def some_inner_function():
      8         value += 1
      9 
     10     some_value = "I don't know what you were expecting"
---> 11     raise UnnecessaryError("You actually called this function...")

ipdb> q


The game seems to be broken as the dice are not added correctly, and an exception gets thrown when we quit playing for some reason. As you can see, the debugger was called once an exception happened. This is shown by the pdb prompt and the additional input rectangle. 

At this point, we haven't discussed how to use pdb. So let's turn it off, but before we need to quit the debugger at the `ipdb>` prompt by typing 'q'.

In [ ]:
%pdb off

With only a few commands, you will be able to achieve a lot in pdb. Here they are:
- l (list)- Display 11 lines from where the current execution line is
- s (step) - Execute one step from the current execution line
- n (next) - Execute one step in the current function without going down in subfunctions
- b (break) ... - set a break point at...
- r (return) - Continue execution in current function until it returns
- c (continue) - Continue the execution until next break point if any, or program termination
- h (help) - Will list all commands. 'help list' will give you help on list
- q (quit) - quit debugger.

Typing the name of a variable will display its value. This means that variables called 'r', or 'l' are out of luck. Remember this when calling your variables in Python. Regardless of the software development environment that you use, these debugging commands are always the same. 

We will now manually set a breakpoint in the code. In another tab, edit file 'Code_08/main.py' and add a line as follows:
```python
from dicegame.runner import GameRunner


def main():
    print("Add the values of the dice")
    print("It's really that easy")
    print("What are you doing with your life.")
    import pdb; pdb.set_trace() # add pdb here
    GameRunner.run()


if __name__ == "__main__":
    main()

```
Now let's run it again... And slowly go through the instructions in the next cell by typing these commands at the `>ipdb` prompt.


In [ ]:
%run Code_08/main.py

The line we added hasn't done much except stop the execution of the script right after the `set_trace()` line, before entering the `run()` function. This allows us to slowly step through the code (using the 's' command). The displayed '->' in the source code shows where the execution is at. One step and we enter the run() function. One more and we land on variable 'c' that is getting initialized. Bad name, as it would conflict with the c(ontinue) command of the debugger if we were to interrogate its value. Its value, however,  can still be displayed using `print(c)` at the debugger's prompt. Now let's use 'n' to avoid going down in the rabbit hole of called sub functions.
- Call 'n' a few more times until you get to the `for` loop on line 31.
- Look at the length of the `for` loop by typing `len(runner.dice)`.
- At this point, we could call 'n' 5 more times, but instead we will set a breakpoint at line 34 using `b 34`.
- Using 'c' (continue) we can execute until this new breakpoint. We just jumped over the `for` loop.
- Breakpoints can be displayed by using the 'b' command without any argument. To clear the break point we just hit, we use the 'clear Num' function on the 'Num' displayed in the leftmost column. Therefore, 'cl 1' would clear the breakpoint we just set.
- We are still at line 34 of the runner.py source file.

- Our script had two issues: (1) It couldn't add dice properly and (2) it threw an exception after quiting to play. Line 37 contains the statement where dice are added. The addition error has to be where the `guess == runner.answer()` equality condition. We 'n' to line 35 where we will need to input our answer. 
- We 'n' again to line 37 where we will step 's' to go into `runner.answer()` and investigate.
- Our first step into `runner.answer()` lands on line 15 on the `answer()` definition. It doesn't look like the `total` variable is correctly adding the values of the dice.
- Jump into the `for` loop on line 18 by either setting a breakpoint and continue, or using next a few times.
- Use `dir(die)` to check whether a `die` has an attibute which would contain its value. 
- We can compare the value of the die with what is shown using `die.value` and `die.show()`. Note that the `\n` characters can be interpreted using `print(die.show())`. Value and die seem to match. The error is that total does not properly add values while it is iterating over the dice. We found one bug and its solution so that you now know how to fix the `answer()` method.

- We can now continue and try to find the other bug. We instruct the debugger to return from the answer() function by typing 'r' and jump to the last line (19) of the answer() function.
- 'total' will display the value of the variable, confirming that the loop is counting dice rather than adding their values.

<div class="alert alert-block alert-info">
    - (Optional) The script contains a few more bugs. Use the `pdb` commands you just learnt to investigate and fix the game.
    <br>- (Optional) File 'Code_08/README.md' contains additional topics such as post-mortem analysis. These methods are helpful for debugging a script after a crash as opposed to interactively as we did.
</div>

